In [1]:
import sys
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis\read")
sys.path.append(r"C:\Users\hjia9\Documents\GitHub\data-analysis")

import numpy as np
import time

from read_scope_data import read_trc_data
from data_analysis_utils import analyze_downsample_options, Photons, hl_envelopes_idx

import matplotlib.pyplot as plt
from scipy.ndimage import gaussian_filter1d
from scipy.signal import fftconvolve, savgol_filter, find_peaks, find_peaks_cwt

%matplotlib qt

In [12]:
ifn = r"F:\x-ray\20241102\C3--E-ring-p24-z13-x40-xray--00036.trc"
xray_data, tarr_x = read_trc_data(ifn)

Reading data...
Done


In [13]:
detector = Photons(tarr_x, xray_data,savgol_window=31, tsh_mult=[9, 100], downsample_rate=8)
                 
detector.reduce_pulses()
pulse_times, pulse_areas = detector.get_pulse_arrays()

Applying Savitzky-Golay filter...
Min distance: 312
Computing baseline...
Computing thresholds...
Detecting pulses...


In [9]:
time_ms = tarr_x * 1000
filtered_xray_data = savgol_filter(xray_data, window_length=31, polyorder=3)

start_idx = 10000000
end_idx = 10500000

plt.figure(0)
plt.plot(time_ms[start_idx:end_idx], xray_data[start_idx:end_idx], label='Original')
plt.plot(time_ms[start_idx:end_idx], filtered_xray_data[start_idx:end_idx], label='Filtered')
plt.legend(loc='upper right')
plt.show()

In [6]:
min_ts = 0.5e-3
# analyze_downsample_options(filtered_xray_data, time_ms, window_length=51, polyorder=3, min_timescale_ms=min_ts)

In [7]:
# Downsample data using recommended rate of 2
downsample_rate = 8

# Create downsampled arrays
time_ds = time_ms[::downsample_rate]
data_ds = filtered_xray_data[::downsample_rate]

# Calculate minimum distance between peaks based on min_timescale
sample_period = (time_ds[1] - time_ds[0])  # Time between samples
min_distance = int(min_ts / sample_period * 10)
print(min_distance)

# Plot comparison
plt.figure(figsize=(12,6))
plt.plot(time_ms, -filtered_xray_data, 'b-', label='Original', alpha=0.5)
plt.plot(time_ds, -data_ds, 'r-', label='Downsampled', linewidth=2)
plt.xlabel('Time (ms)')
plt.ylabel('Signal (V)')
plt.title(f'Original vs Downsampled Data (rate={downsample_rate})')
plt.legend(loc='upper right')
plt.grid(True)
plt.show()

312


In [14]:
larr, harr = hl_envelopes_idx(data_ds, dmin=1, dmax=min_distance, split=False)

plt.figure()
plt.plot(time_ds, data_ds)
plt.plot(time_ds[larr], data_ds[larr], label='Lower Envelope')
plt.plot(time_ds[harr], data_ds[harr], label='Upper Envelope')
plt.xlabel('Time (ms)')
plt.ylabel('Signal')
plt.legend(loc='upper right')
plt.show()

In [15]:
noise_sample = data_ds[:int(len(data_ds)*0.001)]
noise_amplitude = (np.max(np.abs(noise_sample)) - np.min(np.abs(noise_sample))) / 2

# Interpolate baseline using upper envelope points
baseline = np.interp(np.arange(len(data_ds)), harr, data_ds[harr]) - noise_amplitude
baseline_subtracted = data_ds - baseline

plt.figure()
plt.plot(time_ds, data_ds, label='Original')
plt.plot(time_ds, baseline, label='Baseline')
plt.plot(time_ds, baseline_subtracted, label='Baseline Subtracted')
plt.legend(loc='upper right')
plt.show()

In [125]:
data = -baseline_subtracted
# Use first 5% of data as noise sample for threshold determination
noise_sample = data[:int(len(data)*0.05)]
noise_mean = np.mean(noise_sample)
noise_std = np.std(noise_sample)

lower_threshold = noise_mean + 6*noise_std
upper_threshold = noise_mean + 100*noise_std

# Find peaks above lower threshold with minimum distance constraint
peak_indices, _ = find_peaks(data, height=lower_threshold, distance=min_distance)

# Remove peaks that exceed upper threshold and nearby peaks
mask = np.ones(len(peak_indices), dtype=bool)
for i, idx in enumerate(peak_indices):
    if data[idx] > upper_threshold:
        # Find all peaks within min_distance of this large peak
        nearby_mask = np.abs(peak_indices - idx) <= min_distance*20
        mask[nearby_mask] = False

# Apply mask to keep only valid peaks
peak_indices = peak_indices[mask]

# Convert peak indices back to original time coordinates
pulse_times = time_ds[peak_indices]
pulse_amplitudes = data[peak_indices]

plt.figure()
plt.plot(time_ds, data)
plt.axhline(y=lower_threshold, color='g', linestyle='--', label='Lower Threshold')
plt.axhline(y=upper_threshold, color='r', linestyle='--', label='Upper Threshold')
plt.scatter(pulse_times, pulse_amplitudes, color='red')
plt.xlabel('Time (ms)')
plt.ylabel('Signal')
plt.title('Detected Pulses')
plt.legend()
plt.show()


In [5]:
# Create histogram of raw signal data
plt.figure(figsize=(10,6))
plt.hist(xray_data, bins=1000, density=True)
plt.xlabel('Signal Amplitude (V)') 
plt.ylabel('Density')
plt.title('Histogram of X-ray Signal Amplitudes')
plt.grid(True)
plt.show()

In [ ]:
# Create histogram of pulses over time with adjustable bin width
bin_width = 0.1

plt.figure(figsize=(10,6))
bins = np.arange(min(pulse_times), max(pulse_times) + bin_width, bin_width)
hist, bins = np.histogram(pulse_times, bins=bins)
bin_centers = (bins[:-1] + bins[1:])/2

plt.plot(bin_centers, hist)
plt.xlabel('Time (ms)')
plt.ylabel('Counts') 
plt.title(f'Photon Counts vs Time')
plt.grid(True)
plt.show()


In [ ]:
print(min_distance)